In [ ]:
pip install -e ../.

In [ ]:
import gSELECT.io as gsio
import gSELECT.feature_selection as gsfs
import gSELECT.classification as gsc
import gSELECT.visualization as gsv

In [ ]:
filepath = "your/path/here"
output_path = "output"

## Explore the Dataset

Use `explore_h5ad()` to preview the structure of your input file. This helps you choose filters or column names.

In [ ]:
gsio.explore_h5ad(filepath)

## Load Gene Expression Data

You can load data in one of two ways:



### Option 1: Load from `.h5ad` (AnnData)

Use this when working with `.h5ad` files, which often contain metadata like cell types or experimental conditions.

Below are all available options for `load_h5ad`:

| Parameter        | Type    | Default | Description                                                      |
|------------------|---------|---------|------------------------------------------------------------------|
| `file_path`      | str     | —       | Path to the `.h5ad` file                                         |
| `filter_column`  | str     | None    | Column in `.obs` to filter by (e.g., `"cell_type"`)              |
| `filter_values`  | list    | None    | Values within `filter_column` to keep (e.g., `["T cells"]`)      |

**Example usage:**

```python
filter_column = "your_filtercolumn"  # e.g., "cell_type"
filter_values = ['value_zero', 'value_one']

genes, data = gsio.load_h5ad(
    filepath,
    filter_column=filter_column,
    filter_values=filter_values
)
```

**Output:**  
- `genes`: DataFrame of gene names  
- `data`: DataFrame of expression values (samples × genes)


In [ ]:
filter_column = "your_filtercolumn"
filter_values=['value_zero', 'value_one']

genes,data = gsio.load_h5ad(filepath,filter_column=filter_column, filter_values=filter_values)

### Option 2: Load from CSV

You can load gene expression data from a CSV file using the `load` function.  
Below are all available options:

| Parameter      | Type    | Default | Description                                 |
|----------------|---------|---------|---------------------------------------------|
| `path`         | str     | —       | Path to the CSV file                        |
| `n_threads`    | int     | 4       | Number of threads for reading               |
| `use_low_memory` | bool  | True    | Optimize memory usage (recommended for large files) |

**Example usage:**

```python
# Basic usage (default options)
genes, data = gsio.load(filepath)

# Advanced usage (custom options)
genes, data = gsio.load(
    filepath,
    n_threads=8,           # Use 8 threads for faster reading
    use_low_memory=False   # Disable low memory mode if you have enough RAM
)
```

**Output:**  
- `genes`: DataFrame of gene names  
- `data`: DataFrame of expression values (samples × genes)

In [ ]:
genes,data = gsio.load(filepath)

## Optional: Create a Final Hold-Out Test Set

This step is optional to prevent information leakage and avoid circularity in the analysis.

- The dataset is first transposed so that **samples are rows** and **genes are columns**.
- 80% of the samples are randomly selected as the **training set**.
- The remaining 20% are set aside as a **final test set**, which will not be used during mutual information (MI) calculation, gene selection, or model training.
- Both sets are then transposed back to the original format (**samples × genes**).

Why take this extra step?

By removing the test set before computing mutual information and training the model, you ensure that no information from those samples influences feature selection or model development. This eliminates circularity and enables an unbiased final evaluation of model performance.

Only the training data should be used for mutual information scoring, gene selection, and training. The final test data should be reserved exclusively for the last evaluation step.


In [ ]:
data_total = data.transpose()
training_data = data_total.sample(frac=0.8)
test_data = data_total.drop(training_data.index)
training_data = training_data.transpose()
test_data = test_data.transpose()

## Compute Mutual Information Scores

This step calculates **mutual information (MI)** between each gene and the class labels in the training data.  
The MI score reflects how informative each gene is for distinguishing between classes and will be used to rank genes for classification.

You can also provide an **optional exclusion list** of genes (e.g., housekeeping or control genes) that you want to exclude from evaluation and ranking.

In [ ]:
exclusion_list = [
    "example_gene"
]
mutual_info = gsfs.compute_mutual_information(genes, training_data, output_folder=output_path,exclusion_list=exclusion_list)

## Classification Step – Running MLP-based Gene Expression Classifiers

This section runs the core classification logic using a **Multilayer Perceptron (MLP)** neural network.  
The classifier is trained using different gene selection strategies and evaluated across multiple randomized sweeps to assess stability and generalization.

You can control various modes of operation through the provided functions and parameters:

---

### Main Options

| Option                | Description                                                                                   |
|-----------------------|----------------------------------------------------------------------------------------------|
| **Number of sweeps**  | Run repeated training/evaluation cycles for stable performance estimates (`number_sweeps`).   |
| **Test mode**         | Provide explicit `test_data`, or let the code auto-split training data if `test_data=None`.   |
| **Gene selection mode** | <ul><li>`0`: Use the selected gene list directly.</li><li>`1`: Random genes (same size as selected list, for baseline).</li><li>`2`: All non-constant genes (automatically selected).</li></ul> |
| **Panel size**        | Specify the number of genes or a list of panel sizes to test.                                 |
| **Custom gene sets**  | Provide your own list of gene names for targeted classification.                              |
| **Exhaustive/greedy search** | Explore all combinations of top N genes, or use greedy search for large N.             |

---

### Functional Entry Points

| Function Name | Purpose | Typical Input | Output |
|---------------|---------|--------------|--------|
| `run_selected_genes` | Use top N ranked genes based on MI | data, MI, N | results list |
| `run_multiple_gene_selections` | Evaluate multiple panel sizes | data, MI, list of N | results dict |
| `run_with_custom_gene_set` | Use a user-defined gene list | data, gene names, MI | results list |
| `run_explorative_gene_selections` | Exhaustive/greedy search of top N | data, MI, N | results dict |
| `run_explorative_gene_selections_with_custom_set` | Exhaustive search of custom set | data, gene names, MI | results dict |
| `run_all_genes` | Use all non-constant genes | data, MI | results list |

---

### Outputs

Each function returns a collection of results:
- **Test accuracy** (balanced)
- **Train accuracy**
- **Gene selection mode**
- **Number of misclassified test samples**

These outputs can be passed directly to the `gsv.plot_results()`, `gsv.plot_multiple_gene_selections()`, or `gsv.plot_explorative_gene_selections()` functions for visualization and comparison.

---

### Special Case: `training_data=None`

If you set `training_data=None` (or omit it), the code will automatically split your input data into training and test sets internally for each sweep.  
This is useful for quick benchmarking or when you do not have a pre-defined test set.  
**Note:** For reproducible results and unbiased evaluation, it is recommended to manually split your data and reserve a final test set before feature selection and model training.

---

### Additional Notes

- Ensure that mutual information scores and gene indices match your data matrix format (`samples × genes`).
- For large gene sets, use greedy or beam search options to avoid combinatorial explosion.
- You can customize the number of sweeps, panel sizes, and selection strategies to fit your analysis needs.



In [ ]:
results = gsc.run_selected_genes(
    training_data,                # Training data (samples × genes)  (use data if no training data split)
    mutual_info,                  # Mutual information scores
    test_data=test_data,          # Explicit test set (remove option if no training data split)
    number_sweeps=7,              # Custom: run 7 sweeps
    top_n_genes=12,               # Custom: use top 12 genes
    include_random=True,          # Custom: include random gene baseline
    max_iterations=300            # Custom: set MLP max iterations to 300
)
gsv.plot_results(
    results,
    output_folder=output_path,    # Custom: output folder
    dpi=400,                     # Custom: PNG resolution
    save_csv=True,               # Custom: also save CSV summary
    csv_name="selected_genes_results.csv", # Custom: CSV filename
    save_png=True                # Custom: save figure as PNG
)

In [ ]:
gene_selection = [2, 4, 8, 16, 32]        # Custom: test these panel sizes
results = gsc.run_multiple_gene_selections(
    training_data,                        # Training data (samples × genes) (use data if no training data split)
    mutual_info,                          # Mutual information scores
    test_data=test_data,                  # Explicit test set (remove option if no training data split)
    number_sweeps=5,                      # Custom: run 5 sweeps
    gene_selection=gene_selection,        # Custom: panel sizes to test
    max_iterations=250                    # Custom: set MLP max iterations to 250
)
gsv.plot_multiple_gene_selections(
    results,
    output_folder=output_path,            # Custom: output folder
    save_csv=True,                        # Custom: also save CSV summary
)

In [ ]:
gene_list = ["gene1", "gene2"]  # Custom: all genes must exist in the dataset
results = gsc.run_with_custom_gene_set(
    training_data,               # Expression data (samples × genes)  (use data if no training data split)
    gene_list,                   # Custom: user-defined gene list
    mutual_info,                 # Mutual information scores
    test_data=test_data,         # Explicit test set (remove option if no training data split)
    number_sweeps=10,            # Custom: run 10 sweeps
    max_iterations=250           # Custom: set MLP max iterations to 250
)
gsv.plot_results(
    results,
    output_folder=output_path,   # Custom: output folder
    dpi=400,                    # Custom: PNG resolution
    save_csv=True,              # Custom: also save CSV summary
    csv_name="custom_gene_set_results.csv", # Custom: CSV filename
    save_png=True               # Custom: save figure as PNG
)

In [ ]:
results = gsc.run_explorative_gene_selections(
    training_data,                # Training data (samples × genes)  (use data if no training data split)
    mutual_info,                  # Mutual information scores
    test_data=test_data,          # Explicit test set (remove option if no training data split)
    number_sweeps=4,              # Custom: run 4 sweeps
    top_n_genes=5,               # Custom: evaluate top 15 genes
    num_threads=6,                # Custom: use 6 threads for parallel execution
    max_iterations=200,           # Custom: set MLP max iterations to 200
    use_greedy_if_large=True,     # Custom: switch to greedy if top_n_genes is large
    greedy_threshold=12           # Custom: use greedy if more than 12 genes
)
gsv.plot_explorative_gene_selections(
    results,
    top_n=10,                     # Custom: show top 10 gene subsets
    output_folder=output_path,    # Custom: output folder
    cmap_name="Blues",            # Custom: colormap for bars
    show_delta=False,              # Custom: plot delta accuracy
    annotate=True,                # Custom: annotate bars with mean ± std
    dpi=600,                      # Custom: PNG resolution
    csv_name="explorative_gene_subset_rankings.csv", # Custom: CSV filename
    save_csv=True,                # Custom: save ranking CSV
    save_png=True,                # Custom: save figure as PNG
    synonym_prefix="S"            # Custom: prefix for subset codes
)

In [ ]:
gene_list = ["gene1", "gene2"]  # Custom: all genes must exist in the dataset
results = gsc.run_explorative_gene_selections_with_custom_set(
    training_data,                   # Training data (samples × genes) (use data if no training data split)
    gene_list,                       # Custom: user-defined gene list
    mutual_info,                     # Mutual information scores
    test_data=test_data,             # Explicit test set (remove option if no training data split)
    number_sweeps=2,                 # Custom: run 2 sweeps
    num_threads=4,                   # Custom: use 4 threads for parallel execution
    max_iterations=250               # Custom: set MLP max iterations to 250
)
gsv.plot_explorative_gene_selections(
    results,
    top_n=10,                        # Custom: show top 10 gene subsets
    output_folder=output_path,       # Custom: output folder
    cmap_name="Blues",               # Custom: colormap for bars
    show_delta=False,                 # Custom: plot delta accuracy
    annotate=True,                   # Custom: annotate bars with mean ± std
    dpi=500,                         # Custom: PNG resolution
    csv_name="explorative_gene_subset_rankings.csv", # Custom: CSV filename
    save_csv=True,                   # Custom: save ranking CSV
    save_png=True,                   # Custom: save figure as PNG
    synonym_prefix="S"               # Custom: prefix for subset codes
)

In [ ]:
results = gsc.run_all_genes(
    training_data,                # Training data (samples × genes) (use data if no training data split)
    mutual_info,                  # Mutual information scores
    test_data=test_data,          # Explicit test set (remove option if no training data split)
    number_sweeps=3,              # Custom: run 3 sweeps
    max_iterations=500            # Custom: set MLP max iterations to 500
)
gsv.plot_all_genes(
    results,
    output_folder=output_path,    # Custom: output folder
    dpi=600,                      # Custom: PNG resolution
    csv_name="all_genes_rankings.csv", # Custom: CSV filename
    save_csv=True,                # Custom: save summary CSV
    save_png=True                 # Custom: save figure as PNG
)